In [1]:
import json

def slugify(k):
    return k + '-' + str(hash(k))[-4:]

roots = {}
orgs = {}
persons = {}
with open('./data.jl', 'r') as f:
    for l in f.readlines():
        l = l.strip()
        d = json.loads(l)
        if 'roleSlug' in d:
            roots[d['orgSlug']] = [d['roleSlug']]
        elif 'about' in d:
            orgs[d['slug']] = d
        elif 'role' in d:
            persons[d['slug']] = d

res = {'ORGS':{},'PERSONS':{}, 'ROLES':{}}
for k, o in orgs.items():
    res['ORGS'][k] = {
        'slug': o['slug'],
        'name': o['name'],
        'about': o['about'],
        # 'image': o['image'],
        'image': '',
        'ocRootsRoleSlugs': roots.get(o['slug'], []),
        'roleSlugs': []
    }
for k, p in persons.items():
    if p['name']:
        slug = slugify(k)
        orgSlug = p['orgSlug']
        roleSlug = orgSlug + '_' + slug
        res['PERSONS'][slug] = {
            'slug': slug,
            'name': p['name'],
            'roleSlugs': [roleSlug],
            # 'image': p['image'],
            'image': '',
            'social': {
                'linkedin': "",
            },
        }
        res['ORGS'][orgSlug]['roleSlugs'].append(roleSlug)
        directReportsRoleSlugs = []
        reportsToRoleSlugs = []
        if len(p['managerSlug']) > 0:
            reportsToRoleSlugs = [slugify(p['managerSlug'])]
        for s in p['directReportsSlugs']:
            directReportsRoleSlugs.append(slugify(s))
        res['ROLES'][roleSlug] = {
            'slug': roleSlug,
            'orgSlug': orgSlug,
            'personSlug': slug,
            'name': p['role'],
            'directReportsRoleSlugs': directReportsRoleSlugs,
            'reportsToRoleSlugs': reportsToRoleSlugs,
        }

with open('../web/store/res.js', 'w') as js:
    js.write('export const res = ' + str(res))